<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/dynamic_distillation4071.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import solve_ivp

In [66]:
alpha=2.
tray_m = 1.
reboiler_m = 5.
condenser_m = 5.

NT=10
NF=5

weir_constant = 0.1
Kc = 10.

F = 1.
z = 0.6
q = 0.6  #60mol% liquid

D = 0.55
R = 2.

B = F-D

V_rec = (R+1)*D  #Negligible vapor hold up, perfect control on reboiler boil up
V_strip = V_rec - (1-q)*F

x_initial = np.full(NT+2, z)
L_rec_initial = R*D
L_strip_initial = L_rec_initial + q*F

L_initial=np.zeros(NT+2)
L_initial[1:NF+1] = L_strip_initial
L_initial[NF+1:-1] = L_rec_initial

m_initial= np.r_[reboiler_m,       (L_initial[1:-1]/weir_constant)**(2/3)+ tray_m    , condenser_m]
tend=500

In [67]:
m_initial

array([5.        , 7.61148902, 7.61148902, 7.61148902, 7.61148902,
       7.61148902, 5.94608744, 5.94608744, 5.94608744, 5.94608744,
       5.94608744, 5.        ])

In [68]:
def rhs(t, vec, R):
    m, x = np.split(vec,2)
    y=x*alpha/(1-x + x*alpha)
    dm = np.zeros_like(m)
    dmx = np.zeros_like(m)
    L = np.zeros_like(m)
    V = np.zeros_like(m)
    V[:-1] = (R+1)*D
    V[:NF] -=(1-q)*F

    L[1:-1] = weir_constant*(m[1:-1]- tray_m)**1.5
    L[-1] = R*D
    L[0] = B + Kc*(m[0] - reboiler_m)

    dm[1:-1] = L[2:] + V[:-2] - L[1:-1] - V[1:-1]
    dmx[1:-1] = x[2:]*L[2:] + y[:-2]*V[:-2] - x[1:-1]*L[1:-1] - y[1:-1]*V[1:-1]

    dm[NF] += F
    dmx[NF] += z*F

    dm[0] = L[1] - V[0] - L[0]
    dmx[0] = x[1]*L[1] - y[0]*V[0] - x[0]*L[0]

    dm[-1] = V[-2] - L[-1] -  D
    dmx[-1] = y[-2]*V[-2] - x[-1]*L[-1] -  x[-1]*D

    dx =  (dmx - x*dm)/m

    return np.concatenate([dm, dx])

In [71]:
res=solve_ivp(lambda t, v: rhs(t,v,R), (0,tend), np.concatenate([m_initial, x_initial]), method='Radau', dense_output=True)
tplot = np.linspace(0,tend,200)
mplot, xplot=res.sol(tplot).reshape(2,NT+2,200)
fig = make_subplots(rows=1, cols=2)
for i in range(NT+2):
    fig.add_scatter(x=tplot, y= xplot[i,:], mode='lines', row=1,col=1)
    fig.add_scatter(x=tplot, y= mplot[i,:], mode='lines', row=1,col=2)
fig.update_layout(width=1000,height=500,template="plotly_dark")

In [74]:
mcx = xplot[:,-1]
mcy = mcx*alpha/(1-mcx + mcx*alpha)
mcx = np.repeat(mcx,2)
mcy = np.repeat(mcy,2)
mcy[1:]=mcy[:-1]

mc  = np.c_[mcx,mcy]
mc[0,1]=mc[0,0]
mc[-1,1]=mc[-1,0]
xeq = np.linspace(0,1,100)
yeq= xeq*alpha/(1-xeq + xeq*alpha)
fig = make_subplots()
fig.add_scatter(x=xeq, y =yeq, mode='lines')
fig.add_scatter(x=mc[:,0], y =mc[:,1], mode='lines')
fig.add_scatter(x=mc[::2,0], y=mc[::2,1], mode='lines')
fig.update_layout(width=600,height=600,template='plotly_dark')

In [78]:
res2=solve_ivp(lambda t,v: rhs(t,v, 20), (0,tend), res.y[:,-1], method='Radau', dense_output=True)

In [79]:
tplot = np.linspace(0,tend,200)
mplot, xplot=res2.sol(tplot).reshape(2,NT+2,200)
fig = make_subplots(rows=1, cols=2)
for i in range(NT+2):
    fig.add_scatter(x=tplot, y= xplot[i,:], mode='lines', row=1,col=1)
    fig.add_scatter(x=tplot, y= mplot[i,:], mode='lines', row=1,col=2)
fig.update_layout(width=1000,height=500,template="plotly_dark")

In [85]:
mcx = xplot[:,-1]
mcy = mcx*alpha/(1-mcx + mcx*alpha)
mcx = np.repeat(mcx,2)
mcy = np.repeat(mcy,2)
mcy[1:]=mcy[:-1]

mc  = np.c_[mcx,mcy]
mc[0,1]=mc[0,0]
mc[-1,1]=mc[-1,0]
xeq = np.linspace(0,1,100)
yeq= xeq*alpha/(1-xeq + xeq*alpha)
fig = make_subplots()
fig.add_scatter(x=xeq, y =yeq, mode='lines')
fig.add_scatter(x=mc[:,0], y =mc[:,1], mode='lines')
fig.add_scatter(x=mc[::2,0], y=mc[::2,1], mode='lines')
fig.update_layout(width=600,height=600,template='plotly_dark')